In [1]:
#calling the dependencies
from deepface import DeepFace
import cv2
import matplotlib.pyplot as plt

In [3]:
img = cv2.imread("./img/Jeff.jpg")
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
faces = faceCascade.detectMultiScale(gray_img, 1.3, 5)
for (x,y,w,h) in faces:
    roi_gray = gray_img[y:y+h, x:x+w]
    cv2.imwrite("./images/Jeff.jpeg",roi_gray)

### Face Recognition

In [2]:
import glob
import os
import face_recognition

known_face_encodings = []
known_face_names = []
frame_resizing = 1

images_path = glob.glob(os.path.join("./images/", "*.*"))
for img_path in images_path:
    img = cv2.imread(img_path)
    # rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Get the filename only from the initial file path.
    basename = os.path.basename(img_path)
    (filename, ext) = os.path.splitext(basename)
    # Get encoding
    img_encoding = face_recognition.face_encodings(img)[0]

    # Store file name and file encoding
    known_face_encodings.append(img_encoding)
    known_face_names.append(filename)
print("Encoding images loaded")

Encoding images loaded


In [3]:
import numpy as np

def detect_known_faces(frame):
    small_frame = cv2.resize(frame, (0, 0), fx=frame_resizing, fy=frame_resizing)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
        face_names.append(name)

    face_locations = np.array(face_locations)
    face_locations = face_locations / frame_resizing
    return face_locations.astype(int), face_names

In [4]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    face_locations, face_names = detect_known_faces(gray_img)
    for face_loc, name in zip(face_locations, face_names):
        y1, x1, y2, x2 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
        cv2.putText(frame, name, (x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 200), 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 200), 4)
    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

### Facial emotion 

In [6]:
import cv2
import os
import numpy as np
from PIL import Image

faceCascade = cv2.CascadeClassifier("./haarcascade/haarcascade_frontalface_default.xml")

video = cv2.VideoCapture(0)

while True:
    _, frame = video.read()

    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray_img, 1.3, 5)

    # hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # h, s, v = cv2.split(hsv)

    # lim = 255 - 20
    # v[v > lim] = 255
    # v[v <= lim] += 20

    # final_hsv = cv2.merge((h, s, v))
    # frame = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

    try:
        res = DeepFace.analyze(img_path = frame, actions = ["emotion"])['emotion']
    
        emotion = max(zip(res.values(), res.keys()))[1]
        for (x,y,w,h) in faces:
            cv2.putText(frame, emotion, (x+5,y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 200), 4)
    except Exception:
        print("some error")

    cv2.imshow("Capturing", frame)
    key=cv2.waitKey(1)
    if key == ord('q'):
            break
video.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 19ms/step


In [11]:
import cv2
import os
import numpy as np
from PIL import Image

faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

video = cv2.VideoCapture(0)

while True:
    _, frame = video.read()

    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(gray_img, 1.3, 5)

    # res = DeepFace.analyze(img_path = frame, actions = ["emotion"])['emotion']
    
            
    for (x, y, w, h) in faces:
        
        roi_gray = gray_img[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        try:
            res = DeepFace.analyze(img_path = gray_img, actions = ["emotion"])['emotion']
            emotion = max(zip(res.values(), res.keys()))[1]

            
            cv2.putText(frame, emotion, (x+5,y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        except Exception:
            print("some error")

    cv2.imshow("Capturing", frame)
    key=cv2.waitKey(1)
    if key == ord('q'):
            break
video.release()
cv2.destroyAllWindows()

some error
some error
some error
some error
some error
some error


### Image Enhancing

In [ ]:
import cv2
import numpy as np


def readImagesAndTimes():
  
  times = np.array([ 1/30.0, 0.25, 2.5, 15.0 ], dtype=np.float32)
  
  filenames = ["img_0.033.jpg", "img_0.25.jpg", "img_2.5.jpg", "img_15.jpg"]

  images = []
  for filename in filenames:
    im = cv2.imread(filename)
    images.append(im)
  
  return images, times

if __name__ == '__main__':
  # Read images and exposure times
  print("Reading images ... ")

  images, times = readImagesAndTimes()
  
  
  # Align input images
  print("Aligning images ... ")
  alignMTB = cv2.createAlignMTB()
  alignMTB.process(images, images)
  
  # Obtain Camera Response Function (CRF)
  print("Calculating Camera Response Function (CRF) ... ")
  calibrateDebevec = cv2.createCalibrateDebevec()
  responseDebevec = calibrateDebevec.process(images, times)
  
  # Merge images into an HDR linear image
  print("Merging images into one HDR image ... ")
  mergeDebevec = cv2.createMergeDebevec()
  hdrDebevec = mergeDebevec.process(images, times, responseDebevec)
  # Save HDR image.
  cv2.imwrite("hdrDebevec.hdr", hdrDebevec)
  print("saved hdrDebevec.hdr ")
  
  # # Tonemap using Drago's method to obtain 24-bit color image
  print("Tonemaping using Drago's method ... ")
  tonemapDrago = cv2.createTonemapDrago(1.0, 0.7)
  ldrDrago = tonemapDrago.process(hdrDebevec)
  ldrDrago = 3 * ldrDrago
  cv2.imwrite("ldr-Drago.jpg", ldrDrago * 255)
  print("saved ldr-Drago.jpg")
  
  
  # # Tonemap using Reinhard's method to obtain 24-bit color image
  print("Tonemaping using Reinhard's method ... ")
  tonemapReinhard = cv2.createTonemapReinhard(1.5, 0,0,0)
  ldrReinhard = tonemapReinhard.process(hdrDebevec)
  cv2.imwrite("ldr-Reinhard.jpg", ldrReinhard * 255)
  print("saved ldr-Reinhard.jpg")
  
  # # Tonemap using Mantiuk's method to obtain 24-bit color image
  print("Tonemaping using Mantiuk's method ... ")
  tonemapMantiuk = cv2.createTonemapMantiuk(2.2,0.85, 1.2)
  ldrMantiuk = tonemapMantiuk.process(hdrDebevec)
  ldrMantiuk = 3 * ldrMantiuk
  cv2.imwrite("ldr-Mantiuk.jpg", ldrMantiuk * 255)
  print("saved ldr-Mantiuk.jpg")